In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        continue
        #print(os.path.join(dirname, filename))
import random 
# Any results you write to the current directory are saved as output.

In [3]:
#!pip install -U tensorflow-gpu

In [4]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [5]:
import os
print(os.listdir('./input/coic2/flowers/'))

['daisy', '.ipynb_checkpoints', 'dandelion', 'rose', 'sunflower', 'tulip']


In [6]:
import cv2
from PIL import Image
from tqdm import tqdm

In [7]:
x=[]
y=[]
IMG_SIZE=150
FLOWER_DAISY_DIR='./input/coic2/flowers/daisy'
FLOWER_SUNFLOWER_DIR='./input/coic2/flowers/sunflower'
FLOWER_TULIP_DIR='./input/coic2/flowers/tulip'
FLOWER_DANDI_DIR='./input/coic2/flowers/dandelion'
FLOWER_ROSE_DIR='./input/coic2/flowers/rose'
#flower_class_dir=['./input/coic2/flowers/daisy','./input/coic2/flowers/tulip','./input/coic2/flowers/sunflower',
#                  './input/coic2/flowers/dandelion','./input/coic2/flowers/rose']
def make_train_data(name,DIR,x,y):
    for img in tqdm(os.listdir(DIR)):
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        x.append(np.array(img))
        y.append(name)

In [8]:
make_train_data('Daisy',FLOWER_DAISY_DIR,x,y)
make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR,x,y)
make_train_data('Tulip',FLOWER_TULIP_DIR,x,y)
make_train_data('Dandelion',FLOWER_DANDI_DIR,x,y)
make_train_data('Rose',FLOWER_ROSE_DIR,x,y)

100%|██████████| 735/735 [00:01<00:00, 511.38it/s]


In [9]:
print(len(x),len(y))

4086 4086


In [10]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=random.randint(0,len(y))
        ax[i,j].imshow(x[l])
        ax[i,j].set_title('Flower: '+y[l])
        
plt.tight_layout()

In [11]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
le=LabelEncoder()
Y=le.fit_transform(y)
print(Y)
Y=to_categorical(Y,5)
X=np.array(x)
X=X/255

[0 0 0 ... 2 2 2]


In [12]:
print(X.shape)

(4086, 150, 150, 3)


In [13]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [14]:
print(type(x_train))

<class 'numpy.ndarray'>


In [15]:
print(np.random.seed(42))

None


In [16]:
# # modelling starts using a CNN.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Activation,Dropout
from tensorflow.keras.optimizers import Adam,Adadelta

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(5, activation = "softmax"))

In [17]:
from keras.preprocessing.image import ImageDataGenerator
#solve overfitting: https://ithelp.ithome.com.tw/articles/10203371
#explain genator : https://zhuanlan.zhihu.com/p/30197320
generator=ImageDataGenerator(samplewise_std_normalization=False,
                   #zca_whitening=True, complex picture may take long time
                   #zca_epsilon=1e-06,
                   rotation_range=15,
                   width_shift_range=0.1,
                   height_shift_range=0.1,
                   #shear_range=0.0, # shear image axis but fix the y axis
                   zoom_range=0.1,
                   channel_shift_range=0.0,
                   horizontal_flip=True,
                   vertical_flip=False,
                   rescale=None,
                   #fill_mode='wrap',may not suit to this problem
                   dtype='float32')
generator.fit(x_train)

Using TensorFlow backend.


In [18]:
print(type(generator.fit(x_train)),generator.fit(x_train))

<class 'NoneType'> None


In [19]:

model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
model.build(input_shape=[None, 150,150,3])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 32)      2432      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 37, 37, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 96)        55392     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 96)        0

In [21]:
type(x_train)

numpy.ndarray

In [22]:
batch_size=32
epochs=38
History = model.fit_generator(generator.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)
#History=model.fit(x_train, y_train,validation_data = (x_test,y_test), batch_size=32, epochs=10)

Epoch 1/38
102/102 [==============================] - 15s 144ms/step - loss: 1.4999 - accuracy: 0.2593 - val_loss: 1.2374 - val_accuracy: 0.4927
Epoch 2/38
102/102 [==============================] - 13s 132ms/step - loss: 1.1550 - accuracy: 0.4847 - val_loss: 1.0136 - val_accuracy: 0.5868
Epoch 3/38
102/102 [==============================] - 14s 133ms/step - loss: 1.0211 - accuracy: 0.5973 - val_loss: 1.0018 - val_accuracy: 0.5892
Epoch 4/38
102/102 [==============================] - 13s 127ms/step - loss: 0.9641 - accuracy: 0.6061 - val_loss: 0.9827 - val_accuracy: 0.6027
Epoch 5/38
102/102 [==============================] - 13s 131ms/step - loss: 0.8935 - accuracy: 0.6459 - val_loss: 0.8662 - val_accuracy: 0.6455
Epoch 6/38
102/102 [==============================] - 13s 129ms/step - loss: 0.8537 - accuracy: 0.6847 - val_loss: 0.9481 - val_accuracy: 0.6296
Epoch 7/38
102/102 [==============================] - 13s 129ms/step - loss: 0.8321 - accuracy: 0.6789 - val_loss: 0.8209 - val_ac

KeyboardInterrupt: 

In [23]:
TEST_DIR='./input/coic2/unlabel-flowers'
x_final_test=[]
"""
def make_test_data(TEST_DIR,x):
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        x.append(np.array(img))
make_test_data(TEST_DIR,x_final_test)
"""
for i in range(1,176):
    path='./input/coic2/unlabel-flowers/{}.jpg'.format(i)
    pic=cv2.imread(path,cv2.IMREAD_COLOR)
    pic = cv2.resize(pic, (IMG_SIZE,IMG_SIZE))
    x_final_test.append(np.array(pic))
    


In [25]:
"""
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=random.randint(0,len(y))
        ax[i,j].imshow(x_final_test[l])

plt.tight_layout()
"""


'\nfig,ax=plt.subplots(5,2)\nfig.set_size_inches(15,15)\nfor i in range(5):\n    for j in range (2):\n        l=random.randint(0,len(y))\n        ax[i,j].imshow(x_final_test[l])\n\nplt.tight_layout()\n'

In [26]:
#print(x_final_test[0][0])
print(np.array(x_final_test).shape)
x_final_test=np.array(x_final_test).astype('float32')/255
print(type(x_final_test),x_final_test.shape)

(175, 150, 150, 3)
<class 'numpy.ndarray'> (175, 150, 150, 3)


In [27]:
x_final_test.dtype


dtype('float32')

In [28]:
tmp=model.predict(x_final_test)

In [30]:
#daisy=1, dandelion=2, rose=3, sunflower=4, tulip=5.
#print(tmp)

In [31]:
ans=[]
l=[]
for i,pro in enumerate(tmp):
    l=[i+1,pro.argmax()+1]
    ans.append(l)

In [32]:
print(ans[0],ans[1])

[1, 1] [2, 2]


In [33]:
sub =  pd.DataFrame(ans)
sub = sub.rename(index=str, columns={0: "id", 1: "Class"})
sub.to_csv('submission2.csv', index=False)